Check we have connection correct. To test this we call get a FHIR Conformance statement from the server

In [2]:
import requests
from fhirclient.models.fhirinstant import FHIRInstant
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()
import os

fhir_password = os.getenv("FHIR_PASSWORD")
fhir_username = os.getenv("FHIR_USERNAME")
#server = "https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/"
server = os.getenv("FHIR_SERVER")

api_url = server + "metadata"
print(api_url)
response = requests.get(api_url)
#response.json()

http://192.168.1.67/healthconnect/cdr/fhir/r4/metadata


Find a patient. Simple search for a patient named wrexham

In [3]:


api_url = server + "Patient?name=wrexham"
print(api_url)
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient?name=wrexham


{'resourceType': 'Bundle',
 'id': 'eead0b1c-9fae-48ca-8891-dc34d82a8087',
 'type': 'searchset',
 'timestamp': '2025-12-26T05:46:55Z',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient?name=wrexham'}],
 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Patient/18',
   'resource': {'resourceType': 'Patient',
    'address': [{'id': 'qEFQG',
      'line': ['30 PASTURE ROAD', 'BARTON-UPON-HUMBER', 'S HUMBERSIDE'],
      'period': {'start': '2025-06-24'},
      'postalCode': 'DN18 5HU',
      'use': 'home'}],
    'birthDate': '1987-02-18',
    'gender': 'unknown',
    'generalPractitioner': [{'id': 'ZGilw',
      'identifier': {'period': {'start': '2025-03-02'},
       'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
       'value': 'A20047'},
      'reference': 'Organization/1',
      'type': 'Organization'}],
    'id': '18',
    'identifier': [{'system': 'https://fhir.nhs.uk/Id/nhs-number',
      'type':

Now lets find diagnostic reports for this patient.

In [4]:
api_url = server + "DiagnosticReport?patient=18"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Bundle',
 'id': '43187ab1-6827-4112-a496-ee382098831a',
 'type': 'searchset',
 'timestamp': '2025-12-26T05:46:56Z',
 'total': 1,
 'link': [{'relation': 'self',
   'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport?patient=18'}],
 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport/22',
   'resource': {'resourceType': 'DiagnosticReport',
    'basedOn': [{'identifier': {'assigner': {'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
         'value': 'RBS'}},
       'system': 'https://nw-gmsa.github.io/Id/PLAC',
       'type': {'coding': [{'code': 'PLAC',
          'system': 'http://terminology.hl7.org/CodeSystem/v2-0203'}]},
       'value': '1000152867'},
      'type': 'ServiceRequest'}],
    'category': [{'coding': [{'code': 'GE',
        'system': 'http://terminology.hl7.org/CodeSystem/v2-0074'}]}],
    'code': {'coding': [{'code': 'R125_Cardiology',
       'display': 'PANEL: R125.1, R125.2 -

Can also return details about the organisation that created this.

In [5]:
api_url = server + "Organization"
print(api_url)
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

http://192.168.1.67/healthconnect/cdr/fhir/r4/Organization


{'resourceType': 'Bundle',
 'id': '54d53fd2-44d6-490a-8152-6d32966e474d',
 'type': 'searchset',
 'timestamp': '2025-12-26T05:46:56Z',
 'total': 15,
 'link': [{'relation': 'self',
   'url': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Organization'}],
 'entry': [{'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Organization/1',
   'resource': {'resourceType': 'Organization',
    'active': True,
    'identifier': [{'period': {'start': '2022-12-17'},
      'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
      'value': 'A20047'}],
    'id': '1',
    'meta': {'lastUpdated': '2025-12-19T11:08:54Z', 'versionId': '1'}},
   'search': {'mode': 'match'}},
  {'fullUrl': 'http://192.168.1.67/healthconnect/cdr/fhir/r4/Organization/4',
   'resource': {'resourceType': 'Organization',
    'active': True,
    'id': '4',
    'identifier': [{'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
      'value': 'R0A'}],
    'name': 'MANCHESTER UNIVERSITY NHS FOUNDATION TRUST',
    '

Example using python to interpret the json and also using [fhirclient](https://github.com/smart-on-fhir/client-py)

In [6]:
import fhirclient.models.diagnosticreport as dr
import pandas as pd

api_url = server + "DiagnosticReport?_count=50"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response1JSON = response.json()

print(response1JSON['total'])

reports = []
for entry in response1JSON['entry']:
    #print(entry['resource'])
    print(entry['resource']['resourceType'], entry['resource']['issued'] )
    report = dr.DiagnosticReport(entry['resource'])
    reports.append(report)
    for coding in report.code.coding:
        print(coding.code)

8
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
R125.1
DiagnosticReport 2025-10-14T15:59:16+00:00
R125_Cardiology
DiagnosticReport 2025-12-17T12:22:52+00:00
SANGER-Family-follow-up-Rare
R375.1
DiagnosticReport 2025-12-17T12:25:36+00:00
M80_AML_GX
M80.2
DiagnosticReport 2025-12-18T12:14:40+00:00
PCR-FLT3-ITD
M80.18
M80.21
DiagnosticReport 2025-12-06T13:37:35+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-08-14T09:41:16+00:00
R240.1
R240.1


We are aiming at display graphs based on the order and the report.

We can include the order which in FHIR is held in ServiceRequest by including this in the query.

In [13]:
import fhirclient.models.servicerequest as sr

serviceRequests = []
diagnosticReports = []

api_url = server + "DiagnosticReport?_include=DiagnosticReport:based-on"


while True:
    response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
    responseInclude = response.json()

    #print(responseInclude)
    print(responseInclude['total'])
    entry = responseInclude['entry']
    print(len(entry))
    if len(entry) == 0:
        break
    for entry in responseInclude['entry']:

        if entry['resource']['resourceType'] == 'DiagnosticReport':
            report = dr.DiagnosticReport(entry['resource'])
            diagnosticReports.append(report)
        if entry['resource']['resourceType'] == 'ServiceRequest':
            request = sr.ServiceRequest(entry['resource'])
            serviceRequests.append(request)

    print("ServiceRequest = " + str(len(serviceRequests)))
    print("DiagnosticReport = " + str(len(diagnosticReports)))


    found = False
    for link in responseInclude['link']:
        if link['relation'] == 'next':
            api_url = link['url']
            found = True
            print(api_url)
    if found == False:
        break

8
5
ServiceRequest = 0
DiagnosticReport = 5
http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport?_include=DiagnosticReport:based-on&page=2&queryId=4a538aa7-81be-4d50-b09a-e2c1f027c46a
8
5
ServiceRequest = 2
DiagnosticReport = 8
http://192.168.1.67/healthconnect/cdr/fhir/r4/DiagnosticReport?_include=DiagnosticReport:based-on&page=3&queryId=4a538aa7-81be-4d50-b09a-e2c1f027c46a
8
4
ServiceRequest = 6
DiagnosticReport = 8


Process the diaganostic reports

Process the reports, convert some of the objects into other datatypes.

In [8]:
def performer(my_list):
    performr = ""
    if my_list != None:
        for item in my_list:
            performr = item.display
    return performr

def performerCode(my_list):
    performr = None
    if my_list != None:
        for item in my_list:
            performr = item.identifier.value
    return performr

def codeCode(concept):
    code = ""
    for coding in concept.coding:
        code = coding.code

    return code
def codeDisplay(concept):
    code = ""
    for coding in concept.coding:
        code = coding.display

    return code

def issued(issued):
    if issued == None:
        return None
    return issued.isostring

def serviceRequest(my_list):
    sr = None
    if my_list != None:
        for item in my_list:
            if item.reference != None:
                sr = item.reference.replace('ServiceRequest/', '')
    return sr

print(len(diagnosticReports))
dfDR = pd.DataFrame([vars(s) for s in diagnosticReports])

dfDR['performerDisplay'] = dfDR['performer'].apply(performer)
dfDR['performerCode'] = dfDR['performer'].apply(performerCode)
dfDR['codingCode'] = dfDR['code'].apply(codeCode)
dfDR['codingDisplay'] = dfDR['code'].apply(codeDisplay)
dfDR['issuedDate'] = dfDR['issued'].apply(issued)
dfDR['effectiveDate'] = dfDR['effectiveDateTime'].apply(issued)
dfDR['serviceRequestId'] = dfDR['basedOn'].apply(serviceRequest)

dfDiagnosticReport = dfDR[['id','performerDisplay','performerCode','codingCode', 'codingDisplay', 'issuedDate', 'effectiveDate', 'serviceRequestId']]
dfDiagnosticReport

8


,id,performerDisplay,performerCode,codingCode,codingDisplay,issuedDate,effectiveDate,serviceRequestId
0,7,NHS North West Genomics,699X0,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury...",2025-10-14T15:59:16+00:00,None,None
1,16,NHS North West Genomics,699X0,R125.1,Thoracic aortic aneurysm or dissection (WES or...,2025-10-14T15:59:16+00:00,2025-10-14T15:59:16+00:00,15
2,22,NHS North West Genomics,699X0,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury...",2025-10-14T15:59:16+00:00,None,None
3,61,NHS North West Genomics,699X0,R375.1,Family follow-up testing to aid variant interp...,2025-12-17T12:22:52+00:00,2025-12-17T12:22:52+00:00,60
4,75,NHS North West Genomics,699X0,M80.2,"Acute Myeloid Leukaemia, Multi-target NGS pane...",2025-12-17T12:25:36+00:00,2025-12-17T12:25:36+00:00,74
5,88,NHS North West Genomics,699X0,M80.21,"Acute Myeloid Leukaemia, FLT3 TKD hotspot",2025-12-18T12:14:40+00:00,2025-12-18T12:14:40+00:00,87
6,99,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-06T13:37:35+00:00,2025-12-06T13:37:35+00:00,133
7,119,NHS North West Genomics,699X0,R240.1,Diagnostic testing for known mutation(s) (Targ...,2025-08-14T09:41:16+00:00,2025-08-14T09:41:16+00:00,118


Clean up requests

In [9]:
print(len(serviceRequests))

def requester(item):
    performr = None
    if item != None:
        performr = item.display
    return performr

def requesterCode(item):
    performr = None
    if item != None:
        performr = item.identifier.value
    return performr

dfSR = pd.DataFrame([vars(s) for s in serviceRequests])
dfSR['requesterDisplay'] = dfSR['requester'].apply(requester)
dfSR['requesterCode'] = dfSR['requester'].apply(requesterCode)
dfSR['authoredOnDate'] = dfSR['authoredOn'].apply(issued)

dfServiceRequest = dfSR[['id','requesterDisplay','requesterCode','authoredOnDate']]
dfServiceRequest

6


,id,requesterDisplay,requesterCode,authoredOnDate
0,118,None,None,2025-08-05T00:00:00
1,15,Nottingham University Hospitals,RX1,None
2,60,Manchester University NHS Foundation Trust,R0A,2025-05-04T00:00:00
3,74,Manchester University NHS Foundation Trust,R0A,2025-12-06T00:00:00
4,87,Stockport NHS Foundation Trust,RWJ,2025-12-03T00:00:00
5,133,Liverpool Women's NHS Foundation Trust,REP,2025-11-06T00:00:00


Join both dataframes into a single result.

In [10]:
# ... existing code ...
df = pd.merge(
    dfDiagnosticReport,
    dfServiceRequest,
    left_on='serviceRequestId',
    right_on='id',
    how="left",
    indicator=True,
    suffixes=('_dr', '_sr')
)

df

,id_dr,performerDisplay,performerCode,codingCode,codingDisplay,issuedDate,effectiveDate,serviceRequestId,id_sr,requesterDisplay,requesterCode,authoredOnDate,_merge
0,7,NHS North West Genomics,699X0,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury...",2025-10-14T15:59:16+00:00,None,None,NaN,NaN,NaN,NaN,left_only
1,16,NHS North West Genomics,699X0,R125.1,Thoracic aortic aneurysm or dissection (WES or...,2025-10-14T15:59:16+00:00,2025-10-14T15:59:16+00:00,15,15,Nottingham University Hospitals,RX1,None,both
2,22,NHS North West Genomics,699X0,R125_Cardiology,"PANEL: R125.1, R125.2 - Thoracic aortic aneury...",2025-10-14T15:59:16+00:00,None,None,NaN,NaN,NaN,NaN,left_only
3,61,NHS North West Genomics,699X0,R375.1,Family follow-up testing to aid variant interp...,2025-12-17T12:22:52+00:00,2025-12-17T12:22:52+00:00,60,60,Manchester University NHS Foundation Trust,R0A,2025-05-04T00:00:00,both
4,75,NHS North West Genomics,699X0,M80.2,"Acute Myeloid Leukaemia, Multi-target NGS pane...",2025-12-17T12:25:36+00:00,2025-12-17T12:25:36+00:00,74,74,Manchester University NHS Foundation Trust,R0A,2025-12-06T00:00:00,both
5,88,NHS North West Genomics,699X0,M80.21,"Acute Myeloid Leukaemia, FLT3 TKD hotspot",2025-12-18T12:14:40+00:00,2025-12-18T12:14:40+00:00,87,87,Stockport NHS Foundation Trust,RWJ,2025-12-03T00:00:00,both
6,99,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-06T13:37:35+00:00,2025-12-06T13:37:35+00:00,133,133,Liverpool Women's NHS Foundation Trust,REP,2025-11-06T00:00:00,both
7,119,NHS North West Genomics,699X0,R240.1,Diagnostic testing for known mutation(s) (Targ...,2025-08-14T09:41:16+00:00,2025-08-14T09:41:16+00:00,118,118,None,None,2025-08-05T00:00:00,both
